In [31]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [32]:
def dydx(x,y):
    
    y_derivs = np.zeros(2)
    y_derivs[0] = y[1]
    y_derivs[1] = -1*y[0]
    return y_derivs

In [33]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    x_ipoh = xi + 0.5*h
    x_ipo = xi + h
    y_temp = np.zeros(nv)
    
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipo,y_temp)
    k4[:] = h*y_derivs[:]
    
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    return yipo

In [34]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    imax = 10000
    i = 0
    Delta = np.full(nv,2*tol)
    h_step = h
    while(Delta.max()/tol > 1.0):

        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)

        Delta = np.fabs(y_2 - y_11)
        if(Delta.max()/tol > 1.0):
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)

        if(i>=imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
        i+=1
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    return y_2, h_new, h_step

In [39]:
def rk4_mv(dfdx,a,b,y_a,tol):
    xi = a
    yi = y_a.copy()
    h = 1.0e-4 * (b-a)
    imax = 10000
    i = 0
    nv = len(y_a)
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    flag = 1
    
    while(flag):
        yi_new,h_new,h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        h = h_new
        
        if(xi+h_step>b):
            h = b-xi
            yi_new,h_new,h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            flag = 0
        xi += h_step
        yi[:]
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        if(i>=imax):
            print("Maximum iterations reached")
            raise StopIteration("Iteration number =",i)
        i+=1
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)
    if(xi==b):
        flag = 0
    return x,y

In [40]:
a = 0.0
b = 2.0 * np.pi
y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2
tolerance = 1.0e-6
x,y = rk4_mv(dydx,a,b,y_0,tolerance)

i =   1	x = 0.00062832	h = 0.00062832	b=6.28318531
i =   2	x = 0.00188496	h = 0.00125664	b=6.28318531
i =   3	x = 0.00439823	h = 0.00251327	b=6.28318531
i =   4	x = 0.00942478	h = 0.00502655	b=6.28318531
i =   5	x = 0.01947787	h = 0.01005310	b=6.28318531
i =   6	x = 0.03958407	h = 0.02010619	b=6.28318531
i =   7	x = 0.07979645	h = 0.04021239	b=6.28318531
i =   8	x = 0.16022123	h = 0.08042477	b=6.28318531
i =   9	x = 0.32107077	h = 0.16084954	b=6.28318531
i =  10	x = 0.46646873	h = 0.14539796	b=6.28318531
i =  11	x = 0.59958793	h = 0.13311920	b=6.28318531
i =  12	x = 0.73288919	h = 0.13330126	b=6.28318531
i =  13	x = 0.86614501	h = 0.13325582	b=6.28318531
i =  14	x = 0.99941216	h = 0.13326715	b=6.28318531
i =  15	x = 1.13267648	h = 0.13326432	b=6.28318531
i =  16	x = 1.26594151	h = 0.13326503	b=6.28318531
i =  17	x = 1.39920637	h = 0.13326485	b=6.28318531
i =  18	x = 1.53247127	h = 0.13326490	b=6.28318531
i =  19	x = 1.66573615	h = 0.13326489	b=6.28318531
i =  20	x = 1.79900104	h = 0.13